<a href="https://colab.research.google.com/github/pablo-arantes/making-it-rain/blob/main/Protein%2BMembranes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Hello there!**

This is a Jupyter notebook for running Molecular Dynamics (MD) simulations using OpenMM engine and AMBER force field for **Protein** and **Membrane** systems. This notebook is a supplementary material of the paper "***Making it rain: Cloud-based molecular simulations for everyone***" ([link here](https://doi.org/10.1021/acs.jcim.1c00998)) and we encourage you to read it before using this pipeline.

The main goal of this notebook is to demonstrate how to harness the power of cloud-computing to run microsecond-long MD simulations in a cheap and yet feasible fashion.

---

 **This notebook is NOT a standard protocol for MD simulations!** It is just a simple MD pipeline illustrating each step of a simulation protocol.

---
**Bugs**
- If you encounter any bugs, please report the issue to https://github.com/pablo-arantes/making-it-rain/issues

**Acknowledgments**
- We would like to thank the OpenMM team for developing an excellent and open source engine.

- We would like to thank the [LiPyphilic](https://lipyphilic.readthedocs.io/en/latest/index.html) team for developing an excellent and open source python toolkit for the analysis of lipid membrane simulations.

- Thank you to [Orientations of Proteins in Membranes (OPM) database](https://opm.phar.umich.edu/https://glycam.org/#) server team for their amazing web-tool.

- Making-it-rain by **Pablo R. Arantes** ([@pablitoarantes](https://twitter.com/pablitoarantes)), **Marcelo D. Polêto** ([@mdpoleto](https://twitter.com/mdpoleto)), **Conrado Pedebos** ([@ConradoPedebos](https://twitter.com/ConradoPedebos)) and **Rodrigo Ligabue-Braun** ([@ligabue_braun](https://twitter.com/ligabue_braun)).


- Also, credit to [David Koes](https://github.com/dkoes) for his awesome [py3Dmol](https://3dmol.csb.pitt.edu/) plugin.

- For related notebooks see: [Making-it-rain](https://github.com/pablo-arantes/making-it-rain)

In [ ]:
#@title **Install Conda Colab**
#@markdown It will restart the kernel (session), don't worry.
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title **Install dependencies**
#@markdown It will take a few minutes, please, drink a coffee and wait. ;-)
# install dependencies
%%capture
import sys
import tarfile
import os
import subprocess
import sys

commands = [
    "mamba install -c conda-forge ambertools -y",
    "pip -q install py3Dmol",
    "pip install git+https://github.com/pablo-arantes/biopandas",
    "pip install git+https://github.com/amber-md/pytraj",
    "mamba install openmm",
    "mamba install openmmforcefields -c conda-forge -y",
    "pip install --upgrade MDAnalysis",
    "mamba install -c conda-forge pdbfixer -y",
    "mamba install -c conda-forge lipyphilic"
]

for cmd in commands:
    subprocess.run(cmd, shell=True)

#load dependencies
import sys
from biopandas.pdb import PandasPdb
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *
import os
import urllib.request
import numpy as np
import MDAnalysis as mda
import py3Dmol
import pytraj as pt
import platform
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import seaborn as sb
from statistics import mean, stdev
from pytraj import matrix
from matplotlib import colors
from IPython.display import set_matplotlib_formats

## Using Google Drive to store simulation data

Google Colab does not allow users to keep data on their computing nodes. However, we can use Google Drive to read, write, and store our simulations files. Therefore, we suggest to you to:

1.   Create a folder in your own Google Drive and copy the necessary input files there.
2.   Copy the path of your created directory. We will use it below.

In [ ]:
#@title ### **Import Google Drive**
#@markdown Click in the "Run" buttom to make your Google Drive accessible.
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title **Check if you correctly allocated GPU nodes**

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)


---
# **Loading the necessary input files**

At this point, we should have all libraries and dependencies installed.


**Important**: We are going to start with a **PDB_ID** from the [Orientations of Proteins in Membranes (OPM) database](https://opm.phar.umich.edu/https://glycam.org/#). Make sure your PDB_ID is present in the database.The membrane will be added in the XY plane, and the existing protein from OPM database is already oriented and positioned correctly. You can submit your own PDB file, however, it is up to you to select the protein position (should be oriented in Z axis). Lastly, make sure the PDB file points to the correct structure.

Below, you should provide the names of all input files and the pathway of your Google Drive folder containing them.

**Please, don't use spaces in the files and folders names, i.e. MyDrive/protein_membrane and so on.**

In [ ]:
from openmm.app.pdbfile import PDBFile
#@title **Please, provide the necessary input files below for the protein**:
#@markdown **Important:** Run the cell to prepare your protein.

import warnings
warnings.filterwarnings('ignore')
import os
from Bio.PDB import PDBParser, PDBIO, Select
from Bio.PDB import is_aa
import pandas as pd
from pdbfixer import PDBFixer
import urllib.request

Google_Drive_Path = '/content/drive/MyDrive/' #@param {type:"string"}
workDir = Google_Drive_Path

if os.path.exists(os.path.join(workDir)):
  pass
else:
  os.system("mkdir " + str(workDir))

if os.path.exists(os.path.join(workDir, "name_residues_receptor.txt")):
  os.remove(os.path.join(workDir,"name_residues_receptor.txt"))
else:
  pass

receptor = os.path.join(workDir, "receptor.pdb")

Your_own_PDB = "no" #@param ["yes", "no"]

PDB_ID_or_filename = '1a11' #@param {type:"string"}


if Your_own_PDB == "no":
  pdbfn = PDB_ID_or_filename + ".pdb"
  url = 'https://opm-assets.storage.googleapis.com/pdb/' + pdbfn
  outfnm = os.path.join(workDir, pdbfn)
  urllib.request.urlretrieve(url, outfnm)
else:
  outfnm = os.path.join(workDir, PDB_ID_or_filename)

#prepare receptor
ppdb = PandasPdb().read_pdb(outfnm)
ppdb.df['ATOM'] = ppdb.df['ATOM']
# ppdb.df['HETATM'] = ppdb.df['HETATM'][ppdb.df['HETATM']['residue_name'] != 'HOH']
ppdb.df['ATOM'] = ppdb.df['ATOM'][ppdb.df['ATOM']['atom_name'] != 'OXT']
ppdb.df['ATOM']= ppdb.df['ATOM'][ppdb.df['ATOM']['element_symbol'] != 'H']
ppdb.to_pdb(path=receptor, records=['ATOM', 'HETATM'], gz=False, append_newline=True)

fixer = PDBFixer(filename=receptor)
fixer.removeHeterogens()
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(pH=7.4)
PDBFile.writeFile(fixer.topology, fixer.positions, open(receptor, 'w'))

path = '/content/'

def is_het(residue):
    res = residue.id[0]
    return res != " " and res != "W"

def aa(residue):
    res = residue.id[0]
    return res != "W"


class ResidueSelect(Select):
    def __init__(self, chain, residue):
        self.chain = chain
        self.residue = residue

    def accept_chain(self, chain):
        return chain.id == self.chain.id

    def accept_residue(self, residue):
        return residue == self.residue and aa(residue)

def extract_ligands(path):
    pdb = PDBParser().get_structure(receptor, receptor)
    io = PDBIO()
    io.set_structure(pdb)
    i2 = 1
    name_residues2 = []
    for model in pdb:
      for chain in model:
        for residue in chain:
          if not aa(residue):
            continue
          # print(f"{chain[i].resname} {i}")
          name_residues2.append(residue)
          # print((f"saving {residue}"), file=open(os.path.join(workDir, "name_residues_receptor.txt"), "a",))
          i2 += 1

extract_ligands(path)



if os.path.exists(receptor):
  print("Successfully generated the files! :-)")
else:
  print("ERROR: Check your inputs! ")

In [ ]:
#@title **Protein Visualization**:
#@markdown Now the protein has been sanitized, it would be recomended to visualize and check the protein.

view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open(receptor,'r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.3,'color':'white'})
view.zoomTo()
view.show()

In [ ]:
import parmed as pmd
# Imports from the toolkit
import openff.toolkit
from openff.toolkit.topology import Molecule, Topology
# Imports from dependencie
from openmm.app import *
from openmm import *
from openmm.unit import *
from openmm import unit

#@title **Parameters to generate the topology:**

#@markdown The method used here actually adds both a membrane and a water box. It is best to build them together, both to avoid adding waters inside the membrane and to ensure that lipid head groups are properly solvated.

#@markdown The algorithm is based on the one described in [Wolf et al., J. Comp. Chem. 31, pp. 2169-2174 (2010)](https://onlinelibrary.wiley.com/doi/full/10.1002/jcc.21507). It begins by tiling copies of a pre-equilibrated membrane patch to create a membrane of the desired size. Next it scales down the protein by 50% along the X and Y axes. Any lipid within a cutoff distance of the scaled protein is removed. It also ensures that equal numbers of lipids are removed from each leaf of the membrane. Finally, 1000 steps of molecular dynamics are performed to let the membrane relax while the protein is gradually scaled back up to its original size.

#@markdown **Parameters to generate the protein topology:**

Force_field = "AMBER99SB" #@param ["AMBER14SB", "AMBER99SB", "AMBER99SB-ILDN"]
Water_type = "TIP3P" #@param ["TIP3P", "SPC/E"]

#@markdown The size of the membrane and water box are determined by the minimumPadding argument. All pre-existing atoms are guaranteed to be at least this far from any edge of the periodic box. In particular, it only adds whole copies of the pre-equilibrated membrane patch, so the box dimensions will always be integer multiples of the patch size.

minimum_Padding = 1 #@param {type:"slider", min:1, max:10, step:0.5}

#@markdown **ATTENTION**: Give the concentration in Molar units:

Ions = "NaCl" #@param ["NaCl", "KCl" ]
Concentration = "0.15" #@param {type:"string"}

#@markdown **Function to add missing hydrogen atoms based on pH:**

pH = "7.4" #@param {type:"string"}

#@markdown **Parameters to generate the membrane:**

membrane = "POPC" #@param ['POPC', 'POPE', 'DLPC',	'DLPE',	'DMPC',	'DOPC',	'DPPC']

if Force_field == "AMBER14SB" and Water_type == "TIP3P":
  forcefield = ForceField('amber14-all.xml', 'amber14/tip3p.xml')
elif Force_field == "AMBER99SB-ILDN" and Water_type == "TIP3P":
  forcefield = ForceField('amber99sbildn.xml', 'amber14/lipid17.xml', 'tip3p.xml')
elif Force_field == "AMBER99SB" and Water_type == "TIP3P":
  forcefield = ForceField('amber99sb.xml', 'amber14/lipid17.xml', 'tip3p.xml')
elif Force_field == "AMBER14SB" and Water_type == "SPC/E":
  forcefield = ForceField('amber14-all.xml', 'amber14/spce.xml')
elif Force_field == "AMBER99SB-ILDN" and Water_type == "SPC/E":
  forcefield = ForceField('amber99sbildn.xml', 'amber14/lipid17.xml', 'spce.xml')
elif Force_field == "AMBER99SB" and Water_type == "SPC/E":
  forcefield = ForceField('amber99sb.xml', 'amber14/lipid17.xml', 'spce.xml')
else:
  pass


# Load the receptor structure into Modeller
pdb = PDBFile(receptor)
modeller = Modeller(pdb.topology, pdb.positions)

# Add the ligand
# Under the hood, this line uses the OpenFF Toolkit to generate new parameters for the ligand
modeller.addHydrogens(forcefield, pH=float(pH))

if Ions == "NaCl":
  positive_ion = 'Na+'
else:
  positive_ion = 'K+'

# solvate it in 0.15 M NaCl solution
modeller.addMembrane(
    forcefield,
    lipidType=membrane,
    minimumPadding=float(minimum_Padding)*nanometer,
    ionicStrength=float(Concentration) * unit.molar,
    positiveIon=positive_ion, negativeIon='Cl-', neutralize=True
    )

# Retrieve the OpenMM Topology, which stores the atoms and connectivity
topology = modeller.getTopology()

# Get the initial positions
# The box is about 75 angstroms per side, so add (30, 30, 30) to center the protein
positions = modeller.getPositions()

#Export PDB file
system_pdb = os.path.join(workDir, 'system.pdb')
PDBFile.writeFile(topology, positions, open(system_pdb, 'w'))

# ParmEd's GROMACS exporter can't handle constraints from openmm, so we need a variant for export without them
export_system = forcefield.createSystem(
    modeller.topology,
    # constraints=None,
    # rigidWater=False,
    flexibleConstraints=True
)

# Combine the topology, system and positions into a ParmEd Structure
pmd_complex_struct = pmd.openmm.load_topology(topology, export_system, positions)

#AMBER
amber_prmtop = os.path.join(workDir, 'system_amber.prmtop')
amber_inpcrd = os.path.join(workDir, 'system_amber.inpcrd')
pmd_complex_struct.save(amber_prmtop, overwrite=True)
pmd_complex_struct.save(amber_inpcrd, overwrite=True)
pdb_check = os.path.exists(system_pdb)
top_amber = os.path.exists(amber_prmtop)
inpcrd_amber = os.path.exists(amber_inpcrd)
#GROMACS
gromacs_top = os.path.join(workDir, 'system_gromacs.top')
gromacs_gro = os.path.join(workDir, 'system_gromacs.gro')
pmd_complex_struct.save(gromacs_top, overwrite=True)
pmd_complex_struct.save(gromacs_gro, overwrite=True)
top_gromacs = os.path.exists(gromacs_top)
gro_gromacs = os.path.exists(gromacs_gro)
#OpenMM
pmd_complex_struct.save(os.path.join(workDir,'system_openMM.pdb'), overwrite=True)
# with open(os.path.join(workDir,'system_openMM.xml'), 'w') as output:
#   output.write(XmlSerializer.serialize(export_system))
openmm_pdb = os.path.exists(os.path.join(workDir,'system_openMM.pdb'))
# openmm_xml = os.path.exists(os.path.join(workDir,'system_openMM.xml'))

if pdb_check == True and top_amber == True and inpcrd_amber == True and top_gromacs == True and gro_gromacs == True and openmm_pdb == True:
  print("Successfully generated topology! :-)")
else:
  print("ERROR: Check your inputs! ")

## Let's take a look on our simulation box:

In [ ]:
#@title **Show 3D structure**

#@markdown Now the system has been built, it would be recomended to visualize and check the system.
view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open(system_pdb,'r').read(),format='pdb')

Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
lip = ['POP','DLP','DMP','DOP','DPP']
view.addStyle({'and':[{'resn':lip}]},
                       {'stick':{'colorscheme':'greenCarbon','radius':0.2}})
view.setViewStyle({'style':'outline','color':'black','width':0.1})
view.addSurface(py3Dmol.VDW,{'opacity':0.3,'color':'white'})
view.zoomTo()
view.show()

---
---
# **Equilibrating the simulation box - Waters (Part 1)**

Proper MD equilibration protocol is designed to equilibrate both temperature and pressure throughout the simulation box while preserving the protein and membrane experimental conformation. In addition, we also allow the solvent to accomodate around the protein and membrane (Part 1) and solvent and membrane accomodate around the protein (Part 2).

Below, we will set up the MD equilibration parameters, such as temperature, pressure and the desired simulation time. We will define the force constant used to restraint protein heavy-atoms and the membrane atoms in place and the frequency at which we want to save atomic coordinates in a trajectory file (.dcd).

After you are done, you can run the next 2 cells to equilibrate the water in your system.

In [ ]:
#@title ### **Parameters for MD first Equilibration protocol (waters):**

# remove whitespaces
Jobname = 'equil_wat' #@param {type:"string"}

Minimization_steps = "100000" #@param ["1000", "5000", "10000", "20000", "50000", "100000"]

#@markdown Simulation time (in nanoseconds) and integration time (in femtoseconds):
Time = "5" #@param {type:"string"}
stride_time_eq = Time
Integration_timestep = "2" #@param ["0.5", "1", "2", "3", "4"]
dt_eq = Integration_timestep

#@markdown Temperature (in Kelvin) and Pressure (in bar)
Temperature = "310" #@param {type:"string"}
temperature_eq = Temperature
Pressure = 1 #@param {type:"string"}
pressure_eq = Pressure

#@markdown Surface tension acting on the system (in bar):
surface_tension = "0" #@param {type:"string"}

#@markdown Position restraints force constant (in kJ/mol):
Force_constant = 900 #@param {type:"slider", min:0, max:2000, step:100}

#@markdown Frequency to write the trajectory file (in picoseconds):

Write_the_trajectory = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_trajectory_eq = Write_the_trajectory
#@markdown Frequency to write the log file (in picoseconds):

Write_the_log = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_log_eq = Write_the_log


#@markdown ---


In [ ]:
#@title **Runs an Equilibration MD simulation (NPT ensemble)**
#@markdown Now, let's equilibrate our system!

###########################################

from sys import stdout, exit, stderr
import os, math, fnmatch

#############################################
# Defining MD simulation parameters

jobname = os.path.join(workDir, Jobname)
pdbfile = system_pdb
pdb_md = PDBFile(system_pdb)

time_ps = float(Time)*1000
simulation_time = float(time_ps)*picosecond		# in ps
dt = int(dt_eq)*femtosecond
temperature = float(temperature_eq)*kelvin
savcrd_freq = int(write_the_trajectory_eq)*picosecond
print_freq  = int(write_the_log_eq)*picosecond

pressure	= float(pressure_eq)*bar

restraint_fc = int(Force_constant) # kJ/mol

nsteps  = int(simulation_time.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nprint  = int(print_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nsavcrd = int(savcrd_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))

#############################################
# Defining functions to use below:
def backup_old_log(pattern, string):
	result = []
	for root, dirs, files in os.walk("./"):
		for name in files:
			if fnmatch.fnmatch(name, pattern):

				try:
					number = int(name[-2])
					avail = isinstance(number, int)
					#print(name,avail)
					if avail == True:
						result.append(number)
				except:
					pass

	if len(result) > 0:
		maxnumber = max(result)
	else:
		maxnumber = 0

	backup_file = "\#" + string + "." + str(maxnumber + 1) + "#"
	os.system("mv " + string + " " + backup_file)
	return backup_file

def restraints(system, pdb, fc, restraint_array):

	if fc > 0:
		# positional restraints for all heavy-atoms
		posresPROT = CustomExternalForce('k*periodicdistance(x, y, z, x0, y0, z0)^2;')
		posresPROT.addPerParticleParameter('k')
		posresPROT.addPerParticleParameter('x0')
		posresPROT.addPerParticleParameter('y0')
		posresPROT.addPerParticleParameter('z0')

		for atom1 in restraint_array:
			atom1 = int(atom1)

			xpos  = pdb.positions[atom1].value_in_unit(nanometers)[0]
			ypos  = pdb.positions[atom1].value_in_unit(nanometers)[1]
			zpos  = pdb.positions[atom1].value_in_unit(nanometers)[2]

			posresPROT.addParticle(atom1, [fc, xpos, ypos, zpos])

		system.addForce(posresPROT)

	return system
##############################################

#############################################
print("\n> Simulation details:\n")
print("\tJob name = " + jobname)
print("\tPDB file = " + str(pdbfile))

print("\n\tSimulation_time = " + str(simulation_time))
print("\tIntegration timestep = " + str(dt))
print("\tTotal number of steps = " +  str(nsteps))

print("\n\tSave coordinates each " + str(savcrd_freq))
print("\tPrint in log file each " + str(print_freq))

print("\n\tTemperature = " + str(temperature))
print("\tPressure = " + str(pressure))
#############################################

print("\n> Setting the system:\n")

print("\t- Reading topology and structure file...")
print("\t- Creating system and setting parameters...")
nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
ewaldErrorTolerance = 0.0005
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
friction = 1.0

system = forcefield.createSystem(modeller.topology,nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
                                          constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance, removeCMMotion=True)

with open(os.path.join(workDir, 'system_openMM.xml'), 'w') as output:
  output.write(XmlSerializer.serialize(system))

print("\t- Applying restraints. Force Constant = " + str(Force_constant) + "kJ/mol")
pt_system = pt.iterload(pdbfile, pdbfile)
pt_topology = pt_system.top
restraint_array = pt.select_atoms('!(:H*) & !(:HOH) & !(:NA) & !(:CL) & !(:K)', pt_topology)

system = restraints(system, pdb_md, restraint_fc, restraint_array)

print("\t- Applying PBC conditions...")
system.setDefaultPeriodicBoxVectors(*modeller.topology.getPeriodicBoxVectors())
# print(system.getDefaultPeriodicBoxVectors())

print("\t- Setting barostat...")
# system.addForce(MonteCarloBarostat(pressure, temperature))
system.addForce(MonteCarloMembraneBarostat(pressure, int(surface_tension)*bar*nanometer, temperature, MonteCarloMembraneBarostat.XYIsotropic, MonteCarloMembraneBarostat.ZFree))


print("\t- Setting integrator...")
integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(topology, system, integrator)
simulation.context.setPositions(modeller.positions)
# if pdb_md.boxVectors is not None:
#     simulation.context.setPeriodicBoxVectors(*modeller.boxVectors)

print("\t- Energy minimization: " + str(Minimization_steps) + " steps")
simulation.minimizeEnergy(tolerance=10*kilojoule/mole/nanometer, maxIterations=int(Minimization_steps))

print("\t-> Potential Energy = " + str(simulation.context.getState(getEnergy=True).getPotentialEnergy()))

print("\t- Setting initial velocities...")
simulation.context.setVelocitiesToTemperature(temperature)

#############################################
# Running Equilibration on NPT ensemble

dcd_file = jobname + ".dcd"
log_file = jobname + ".log"
rst_file = jobname + ".rst"
prv_rst_file = jobname + ".rst"
pdb_file = jobname + ".pdb"

# Creating a trajectory file and reporters
dcd = DCDReporter(dcd_file, nsavcrd)
firstdcdstep = (nsteps) + nsavcrd
dcd._dcd = DCDFile(dcd._out, topology, simulation.integrator.getStepSize(), firstdcdstep, nsavcrd) # charmm doesn't like first step to be 0

simulation.reporters.append(dcd)
simulation.reporters.append(StateDataReporter(stdout, nprint, step=True, speed=True, progress=True, totalSteps=nsteps, remainingTime=True, separator='\t\t'))
simulation.reporters.append(StateDataReporter(log_file, nprint, step=True, kineticEnergy=True, potentialEnergy=True, totalEnergy=True, temperature=True, volume=True, speed=True))

print("\n> Simulating " + str(nsteps) + " steps...")
simulation.step(nsteps)

simulation.reporters.clear() # remove all reporters so the next iteration don't trigger them.


##################################
# Writing last frame information of stride
print("\n> Writing state file (" + str(rst_file) + ")...")
state = simulation.context.getState( getPositions=True, getVelocities=True )
with open(rst_file, 'w') as f:
	f.write(XmlSerializer.serialize(state))

last_frame = int(nsteps/nsavcrd)
print("> Writing coordinate file (" + str(pdb_file) + ", frame = " + str(last_frame) + ")...")
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open(pdb_file, 'w'))

print("\n> Finished!\n")

---
---
# **Equilibrating the simulation box - Waters and Membrane (Part 2)**
 Here, we also allow the solvent and membrane accomodate around the protein (Part 2). Note that we will use here a *.rst state file* , which contains atomic velocities and positions from the last frame of the first part of  equilibration simulation, guaranteeing that our second equilibration begins from our previous equilibrated system.

Below, we will set up the MD equilibration parameters, such as temperature, pressure and the desired simulation time. We will define the force constant used to restraint protein heavy-atoms in place and the frequency at which we want to save atomic coordinates in a trajectory file (.dcd).

After you are done, you can run the next 2 cells to equilibrate the water and membrane in your system.

In [ ]:

#@markdown ### **Provide input file names below:**

Equilibrated_PDB = 'equil_wat.pdb' #@param {type:"string"}
State_file = 'equil_wat.rst' #@param {type:"string"}
#@markdown ### **Parameters for MD the second Equilibration protocol (waters and membrane):**

# remove whitespaces
Jobname = 'equil_wat_memb' #@param {type:"string"}

#@markdown Simulation time (in nanoseconds) and integration time (in femtoseconds):
Time = "5" #@param {type:"string"}
stride_time_eq = Time
Integration_timestep = "2" #@param ["0.5", "1", "2", "3", "4"]
dt_eq = Integration_timestep

#@markdown Temperature (in Kelvin) and Pressure (in bar)
Temperature = "310" #@param {type:"string"}
temperature_eq = Temperature
Pressure = 1 #@param {type:"string"}
pressure_eq = Pressure

#@markdown Surface tension acting on the system (in bar):
surface_tension = "0" #@param {type:"string"}

#@markdown Position restraints force constant (in kJ/mol):
Force_constant = 900 #@param {type:"slider", min:0, max:2000, step:100}

#@markdown Frequency to write the trajectory file (in picoseconds):

Write_the_trajectory = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_trajectory_eq = Write_the_trajectory
#@markdown Frequency to write the log file (in picoseconds):

Write_the_log = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_log_eq = Write_the_log


#@markdown ---


In [ ]:
#@title **Runs an Equilibration MD simulation (NPT ensemble)**
#@markdown Now, let's equilibrate our system!

###########################################

from sys import stdout, exit, stderr
import os, math, fnmatch

#############################################
# Defining MD simulation parameters

jobname = os.path.join(workDir, Jobname)
pdbfile = os.path.join(workDir, Equilibrated_PDB)
equil_rst_file = os.path.join(workDir, State_file)
pdb_md = PDBFile(system_pdb)

time_ps = float(Time)*1000
simulation_time = float(time_ps)*picosecond		# in ps
dt = int(dt_eq)*femtosecond
temperature = float(temperature_eq)*kelvin
savcrd_freq = int(write_the_trajectory_eq)*picosecond
print_freq  = int(write_the_log_eq)*picosecond

pressure	= float(pressure_eq)*bar

restraint_fc = int(Force_constant) # kJ/mol

nsteps  = int(simulation_time.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nprint  = int(print_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nsavcrd = int(savcrd_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))

#############################################
# Defining functions to use below:
def backup_old_log(pattern, string):
	result = []
	for root, dirs, files in os.walk("./"):
		for name in files:
			if fnmatch.fnmatch(name, pattern):

				try:
					number = int(name[-2])
					avail = isinstance(number, int)
					#print(name,avail)
					if avail == True:
						result.append(number)
				except:
					pass

	if len(result) > 0:
		maxnumber = max(result)
	else:
		maxnumber = 0

	backup_file = "\#" + string + "." + str(maxnumber + 1) + "#"
	os.system("mv " + string + " " + backup_file)
	return backup_file

def get_cubic_box(system, rstfile):

	f = open(rstfile, 'r')

	box = {}

	while True:
		line = f.readline()
		if not line: break

		if line.split()[0] == "<A":
			size = line.split()[1].strip('x="')
			box['A'] = float(size)
		elif line.split()[0] == "<B":
			size = line.split()[2].strip('y="')
			box['B'] = float(size)
		elif line.split()[0] == "<C":
			size = line.split()[3].strip('z="').strip('"/>')
			box['C'] = float(size)
		else:
			pass

	boxX = [box['A']*nanometer, 0, 0]
	boxY = [0, box['B']*nanometer, 0]
	boxZ = [0, 0, box['C']*nanometer]

	system.setDefaultPeriodicBoxVectors(boxX, boxY, boxZ)

	return system

def restraints(system, pdb, fc, restraint_array):

	if fc > 0:
		# positional restraints for all heavy-atoms
		posresPROT = CustomExternalForce('k*periodicdistance(x, y, z, x0, y0, z0)^2;')
		posresPROT.addPerParticleParameter('k')
		posresPROT.addPerParticleParameter('x0')
		posresPROT.addPerParticleParameter('y0')
		posresPROT.addPerParticleParameter('z0')

		for atom1 in restraint_array:
			atom1 = int(atom1)

			xpos  = pdb.positions[atom1].value_in_unit(nanometers)[0]
			ypos  = pdb.positions[atom1].value_in_unit(nanometers)[1]
			zpos  = pdb.positions[atom1].value_in_unit(nanometers)[2]

			posresPROT.addParticle(atom1, [fc, xpos, ypos, zpos])

		system.addForce(posresPROT)

	return system
##############################################

#############################################
print("\n> Simulation details:\n")
print("\tJob name = " + jobname)
print("\tPDB file = " + str(pdbfile))

print("\n\tSimulation_time = " + str(simulation_time))
print("\tIntegration timestep = " + str(dt))
print("\tTotal number of steps = " +  str(nsteps))

print("\n\tSave coordinates each " + str(savcrd_freq))
print("\tPrint in log file each " + str(print_freq))

print("\n\tTemperature = " + str(temperature))
print("\tPressure = " + str(pressure))
#############################################

print("\n> Setting the system:\n")

print("\t- Reading topology and structure file...")
print("\t- Creating system and setting parameters...")
nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
ewaldErrorTolerance = 0.0005
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
friction = 1.0

system = forcefield.createSystem(modeller.topology,nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
                                          constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance, removeCMMotion=True)

print("\t- Applying restraints. Force Constant = " + str(Force_constant) + "kJ/mol")
pt_system = pt.iterload(pdbfile, pdbfile)
pt_topology = pt_system.top
restraint_array = pt.select_atoms('!(:H*) & !(:HOH) & !(:NA) & !(:CL) & !(:K) & !(:DLP) & !(:DMP) & !(:DOP)	& !(:DPP) & !(:POP)', pt_topology)

system = restraints(system, pdb_md, restraint_fc, restraint_array)

print("\t- Applying PBC conditions...")
# system.setDefaultPeriodicBoxVectors(*modeller.topology.getPeriodicBoxVectors())
system = get_cubic_box(system,equil_rst_file)
# print(system.getDefaultPeriodicBoxVectors())

print("\t- Setting barostat...")
# system.addForce(MonteCarloBarostat(pressure, temperature))
system.addForce(MonteCarloMembraneBarostat(pressure, int(surface_tension)*bar*nanometer, temperature, MonteCarloMembraneBarostat.XYIsotropic, MonteCarloMembraneBarostat.ZFree))


print("\t- Setting integrator...")
integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(topology, system, integrator)
simulation.context.setPositions(modeller.positions)
# if pdb_md.boxVectors is not None:
#     simulation.context.setPeriodicBoxVectors(*modeller.boxVectors)

#############################################
# Running Equilibration on NPT ensemble

dcd_file = jobname + ".dcd"
log_file = jobname + ".log"
rst_file = jobname + ".rst"
prv_rst_file = jobname + ".rst"
pdb_file = jobname + ".pdb"

print("\n> Loading previous state from equilibration > " + equil_rst_file + " <")
with open(equil_rst_file, 'r') as f:
	simulation.context.setState(XmlSerializer.deserialize(f.read()))
	currstep = int((1-1)*nsteps)
	currtime = currstep*dt.in_units_of(picosecond)
	simulation.currentStep = currstep
	simulation.context.setTime(currtime)
	print("> Current time: " + str(currtime) + " (Step = " + str(currstep) + ")")

# Creating a trajectory file and reporters

dcd = DCDReporter(dcd_file, nsavcrd)
firstdcdstep = (currstep) + nsavcrd
dcd._dcd = DCDFile(dcd._out, topology, simulation.integrator.getStepSize(), firstdcdstep, nsavcrd)

simulation.reporters.append(dcd)
simulation.reporters.append(StateDataReporter(stdout, nprint, step=True, speed=True, progress=True, totalSteps=nsteps, remainingTime=True, separator='\t\t'))
simulation.reporters.append(StateDataReporter(log_file, nprint, step=True, kineticEnergy=True, potentialEnergy=True, totalEnergy=True, temperature=True, volume=True, speed=True))

print("\n> Simulating " + str(nsteps) + " steps...")
simulation.step(nsteps)

simulation.reporters.clear() # remove all reporters so the next iteration don't trigger them.


##################################
# Writing last frame information of stride
print("\n> Writing state file (" + str(rst_file) + ")...")
state = simulation.context.getState( getPositions=True, getVelocities=True )
with open(rst_file, 'w') as f:
	f.write(XmlSerializer.serialize(state))

last_frame = int(nsteps/nsavcrd)
print("> Writing coordinate file (" + str(pdb_file) + ", frame = " + str(last_frame) + ")...")
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open(pdb_file, 'w'))

print("\n> Finished!\n")

---
---
# **Running a Production MD simulation**

Finally, we will proceed with the Production simulation itself using the equilibrated system coordinates as input structure.

Note that we will use here a *.rst state file* , which contains atomic velocities and positions from the last frame of the second part of  equilibration simulation, guaranteeing that our production simulation begins from a thermodynamically equilibrated system.

Another important information here is the **Number_of_strides** and the **Stride_Time**. In this notebook, we simulate a defined number of *strides*, so the **simulation time = Number_of_strides*Stride_Time**. For example, we can simulate 100ns by setting *Number_of_strides=10* and *Stride_Time=10 ns*.

**Important: at the end of the Production simulation, we concatenate all strides to create a complete trajectory file which can be visualized and analyzed**

The idea behind this approach is to make use of the intermitent 12h/24h period that Google Colab allows us to use its GPUs.

In [ ]:
#@markdown ### **Provide input file names below:**

Equilibrated_PDB = 'equil_wat_memb.pdb' #@param {type:"string"}
State_file = 'equil_wat_memb.rst' #@param {type:"string"}

#@markdown ---
#@markdown ### **Parameters for MD Production protocol:**


# remove whitespaces
Jobname = 'prod_wat_memb' #@param {type:"string"}

#@markdown Simulation time (in nanoseconds), number of strides (integers) and integration timestep (in femtoseconds):
Stride_Time = "10" #@param {type:"string"}
stride_time_prod = Stride_Time
Number_of_strides = "10" #@param {type:"string"}
nstride = Number_of_strides
Integration_timestep = "2" #@param ["0.5", "1", "2", "3", "4"]
dt_prod = Integration_timestep

#@markdown Temperature (in Kelvin) and Pressure (in bar)
Temperature = "310" #@param {type:"string"}
temperature_prod = Temperature
Pressure = 1 #@param {type:"string"}
pressure_prod = Pressure

#@markdown Surface tension acting on the system (in bar):
surface_tension = "0" #@param {type:"string"}

#@markdown Frequency to write the trajectory file (in picoseconds):
Write_the_trajectory = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_trajectory_prod = Write_the_trajectory

#@markdown Frequency to write the log file (in picoseconds):
Write_the_log = "10" #@param ["10", "100", "200", "500", "1000"]
write_the_log_prod = Write_the_log
continue_simulation = "no" #@param ["yes", "no"]

#@markdown **Important:** If you are going to continue your simulation, all the parameters should be the same, including the **Equilibrated PDB**, **State file** and **Jobname**. The only parameter you should change is the **Number of strides**, which you will increase.

#@markdown ---

In [ ]:
#@title **Runs a Production MD simulation (NPT ensemble) after second equilibration step**
#
###########################################
from sys import stdout, exit, stderr
import os, math, fnmatch

#############################################
# Defining MD simulation parameters

jobname = os.path.join(workDir, str(Jobname))
pdbfile = os.path.join(workDir, Equilibrated_PDB)
equil_rst_file = os.path.join(workDir, State_file)
pdb_md = PDBFile(pdbfile)


stride_time_ps = float(stride_time_prod)*1000
stride_time = float(stride_time_ps)*picosecond
nstride = int(Number_of_strides)
dt = int(dt_prod)*femtosecond
temperature = float(temperature_prod)*kelvin
savcrd_freq = int(write_the_trajectory_prod)*picosecond
print_freq  = int(write_the_log_prod)*picosecond

pressure	= float(pressure_prod)*bar

simulation_time = stride_time*nstride
nsteps  = int(stride_time.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nprint  = int(print_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nsavcrd = int(savcrd_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
firststride = 1 # must be integer
#############################################
# Defining functions to use below:
def backup_old_log(pattern, string):
	result = []
	for root, dirs, files in os.walk("./"):
		for name in files:
			if fnmatch.fnmatch(name, pattern):

				try:
					number = int(name[-2])
					avail = isinstance(number, int)
					#print(name,avail)
					if avail == True:
						result.append(number)
				except:
					pass

	if len(result) > 0:
		maxnumber = max(result)
	else:
		maxnumber = 0

	backup_file = "\#" + string + "." + str(maxnumber + 1) + "#"
	os.system("mv " + string + " " + backup_file)
	return backup_file

def get_cubic_box(system, rstfile):

	f = open(rstfile, 'r')

	box = {}

	while True:
		line = f.readline()
		if not line: break

		if line.split()[0] == "<A":
			size = line.split()[1].strip('x="')
			box['A'] = float(size)
		elif line.split()[0] == "<B":
			size = line.split()[2].strip('y="')
			box['B'] = float(size)
		elif line.split()[0] == "<C":
			size = line.split()[3].strip('z="').strip('"/>')
			box['C'] = float(size)
		else:
			pass

	boxX = [box['A']*nanometer, 0, 0]
	boxY = [0, box['B']*nanometer, 0]
	boxZ = [0, 0, box['C']*nanometer]

	system.setDefaultPeriodicBoxVectors(boxX, boxY, boxZ)

	return system

##############################################

#############################################
print("\n> Simulation details:\n")
print("\tJob name = " + jobname)
# print("\tPDB file = " + str(pdbfile))

print("\n\tSimulation_time = " + str(stride_time*nstride))
print("\tIntegration timestep = " + str(dt))
print("\tTotal number of steps = " +  str(nsteps*nstride))
print("\tNumber of strides = " + str(nstride) + " (" + str(stride_time) + " in each stride)")

print("\n\tSave coordinates each " + str(savcrd_freq))
print("\tSave checkpoint each " + str(savcrd_freq))
print("\tPrint in log file each " + str(print_freq))

print("\n\tTemperature = " + str(temperature))
print("\tPressure = " + str(pressure))
#############################################

print("\n> Setting the system:\n")


print("\t- Reading topology and structure file...")
print("\t- Creating system and setting parameters...")
nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
ewaldErrorTolerance = 0.0005
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
friction = 1.0

if continue_simulation == "yes":
	for i in range(1,int(nstride)+1):
		name = str(jobname)+ "_" + str(i) + ".pdb"
		name2 = str(jobname) + "_" + str(i) + ".rst"
		if os.path.exists(name) and os.path.exists(name2):
			last_stride_pdb = name
			last_stride_rst = name2
		else:
			pass
	pdb_md = PDBFile(last_stride_pdb)
	with open(os.path.join(workDir, 'system_openMM.xml')) as input:
  		system = XmlSerializer.deserialize(input.read())
	topology = pdb_md.getTopology()
	print("\t- Applying PBC conditions...")
	system = get_cubic_box(system,last_stride_rst)
else:
	system = forcefield.createSystem(modeller.topology,nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
                                          constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance, removeCMMotion=True)
	print("\t- Applying PBC conditions...")
	system = get_cubic_box(system,equil_rst_file)

print("\t- Setting barostat...")
system.addForce(MonteCarloMembraneBarostat(pressure, int(surface_tension)*bar*nanometer, temperature, MonteCarloMembraneBarostat.XYIsotropic, MonteCarloMembraneBarostat.ZFree))

print("\t- Setting integrator...")
integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(topology, system, integrator)
if continue_simulation == "yes":
	simulation.context.setPositions(pdb_md.positions)
else:
	simulation.context.setPositions(modeller.positions)
#if inpcrd.boxVectors is not None:
# 	simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

#############################################
# Opening a loop of extension NSTRIDE to simulate the entire STRIDE_TIME*NSTRIDE
for n in range(1, nstride + 1):

	print("\n\n>>> Simulating Stride #" + str(n) + " <<<")

	dcd_file = jobname + "_" + str(n) + ".dcd"
	log_file = jobname + "_" + str(n) + ".log"
	rst_file = jobname + "_" + str(n) + ".rst"
	prv_rst_file = jobname + "_" + str(n-1) + ".rst"
	pdb_file = jobname + "_" + str(n) + ".pdb"

	if os.path.exists(rst_file):
		print("> Stride #" + str(n) + " finished (" + rst_file + " present). Moving to next stride... <")
		continue

	if n == 1:
		print("\n> Loading previous state from equilibration > " + equil_rst_file + " <")
		with open(equil_rst_file, 'r') as f:
			simulation.context.setState(XmlSerializer.deserialize(f.read()))
			currstep = int((n-1)*nsteps)
			currtime = currstep*dt.in_units_of(picosecond)
			simulation.currentStep = currstep
			simulation.context.setTime(currtime)
			print("> Current time: " + str(currtime) + " (Step = " + str(currstep) + ")")

	else:
		print("> Loading previous state from > " + prv_rst_file + " <")
		with open(prv_rst_file, 'r') as f:
			simulation.context.setState(XmlSerializer.deserialize(f.read()))
			currstep = int((n-1)*nsteps)
			currtime = currstep*dt.in_units_of(picosecond)
			simulation.currentStep = currstep
			simulation.context.setTime(currtime)
			print("> Current time: " + str(currtime) + " (Step = " + str(currstep) + ")")


	dcd = DCDReporter(dcd_file, nsavcrd)
	firstdcdstep = (currstep) + nsavcrd
	dcd._dcd = DCDFile(dcd._out, topology, simulation.integrator.getStepSize(), firstdcdstep, nsavcrd) # first step should not be 0

	simulation.reporters.append(dcd)
	simulation.reporters.append(StateDataReporter(stdout, nprint, step=True, speed=True, progress=True, totalSteps=(nsteps*nstride), remainingTime=True, separator='\t\t'))
	simulation.reporters.append(StateDataReporter(log_file, nprint, step=True, kineticEnergy=True, potentialEnergy=True, totalEnergy=True, temperature=True, volume=True, speed=True))

	print("\n> Simulating " + str(nsteps) + " steps... (Stride #" + str(n) + ")")
	simulation.step(nsteps)

	simulation.reporters.clear() # remove all reporters so the next iteration don't trigger them.


	##################################
	# Writing last frame information of stride
	print("\n> Writing state file (" + str(rst_file) + ")...")
	state = simulation.context.getState( getPositions=True, getVelocities=True )
	with open(rst_file, 'w') as f:
		f.write(XmlSerializer.serialize(state))

	last_frame = int(nsteps/nsavcrd)
	print("> Writing coordinate file (" + str(pdb_file) + ", frame = " + str(last_frame) + ")...")
	positions = simulation.context.getState(getPositions=True).getPositions()
	PDBFile.writeFile(simulation.topology, positions, open(pdb_file, 'w'))

print("\n> Finished!\n")

In [ ]:
#@title **Concatenate and align the trajectory**
#@markdown **Important**: The **Google Drive Path**, **Jobname**, **Number of strides**, **stride time** and **trajectory saved frequency** should be the same you have been used to run your simulation in the previous steps.

import MDAnalysis as mda
from MDAnalysis.analysis import align, rms

Google_Drive_Path = '/content/drive/MyDrive/' #@param {type:"string"}
workDir = Google_Drive_Path
Equilibrated_PDB = 'equil_wat_memb.pdb' #@param {type:"string"}
Jobname = "prod_wat_memb" #@param {type: "string"}
Skip = "1" #@param ["1", "2", "5", "10", "20", "50"]
stride_traj = Skip
Output_format = "dcd" #@param ["dcd", "pdb", "trr", "xtc"]
first_stride = "1" #@param {type:"string"}
Number_of_strides = "10" #@param {type:"string"}
nstride = int(Number_of_strides)
stride_time = "10" #@param {type:"string"}
trajectory_saved_frequency = "10" #@param ["10", "100", "200", "500", "1000"]
traj_save_freq = trajectory_saved_frequency
membrane = "POPC" #@param ['POPC', 'POPE', 'DLPC',	'DLPE',	'DMPC',	'DOPC',	'DPPC']
Remove_waters = "yes" #@param ["yes", "no"]
# stride_id_as_ref_for_alignment = "1" #@param {type: "string"}
output_prefix = first_stride+"-"+str(int(first_stride)+nstride-1)

if membrane == "POPE":
  leaf = 'resname POP'
  polar = 'resname POP and name P O11 O12 O13 O14'
  apolar = 'resname POP and name C* and not name C1 C2 C3 C11 C12'
  polar_density =':POP@P,O11,O12,O13,O14'
  apolar_density = ':POP@C*&!@C1,C2,C3,C11,C12'
  protein_density = '!:POP,NA,CL,K'
elif membrane == "POPC":
  leaf = 'resname POP'
  polar = 'resname POP and name P O11 O12 O13 O14'
  apolar = 'resname POP and name C* and not name C1 C2 C3 C11 C12 C13 C14 C15'
  polar_density =':POP@P,O11,O12,O13,O14'
  apolar_density = ':POP@C*&!@C1,C2,C3,C11,C12,C13,C14,C15'
  protein_density = '!:POP,NA,CL,K'
elif membrane == "DLPC":
  leaf = 'resname DLP'
  polar = 'resname DLP and name P O11 O12 O13 O14'
  apolar = 'resname DLP and name C* and not name C1 C2 C3 C11 C12 C13 C14 C15'
  polar_density =':DLP@P,O11,O12,O13,O14'
  apolar_density = ':DLP@C*&!@C1,C2,C3,C11,C12,C13,C14,C15'
  protein_density = '!:DLP,NA,CL,K'
elif membrane == "DLPE":
  leaf = 'resname DLP'
  polar = 'resname DLP and name P O11 O12 O13 O14'
  apolar = 'resname DLP and name C* and not name C1 C2 C3 C11 C12'
  polar_density =':DLP@P,O11,O12,O13,O14'
  apolar_density = ':DLP@C*&!@C1,C2,C3,C11,C12'
  protein_density = '!:DLP,NA,CL,K'
elif membrane == "DMPC":
  leaf = 'resname DMP'
  polar = 'resname DMP and name P O11 O12 O13 O14'
  apolar = 'resname DMP and name C* and not name C1 C2 C3 C11 C12 C13 C14 C15'
  polar_density =':DMP@P,O11,O12,O13,O14'
  apolar_density = ':DMP@C*&!@C1,C2,C3,C11,C12,C13,C14,C15'
  protein_density = '!:DMP,NA,CL,K'
elif membrane == "DOPC":
  leaf = 'resname DOP'
  polar = 'resname DOP and name P O11 O12 O13 O14'
  apolar = 'resname DOP and name C* and not name C1 C2 C3 C11 C12 C13 C14 C15'
  polar_density =':DOP@P,O11,O12,O13,O14'
  apolar_density = ':DOP@C*&!@C1,C2,C3,C11,C12,C13,C14,C15'
  protein_density = '!:DOP,NA,CL,K'
elif membrane == "DPPC":
  leaf = 'resname DPP'
  polar = 'resname DPP and name P O11 O12 O13 O14'
  apolar = 'resname DPP and name C* and not name C1 C2 C3 C11 C12 C13 C14 C15'
  polar_density =':DPP@P,O11,O12,O13,O14'
  apolar_density = ':DPP@C*&!@C1,C2,C3,C11,C12,C13,C14,C15'
  protein_density = '!:DPP,NA,CL,K'
else:
  pass

stride_time_ps = float(stride_time)*1000
simulation_time_analysis = stride_time_ps*nstride
simulation_ns = float(stride_time)*int(Number_of_strides)
number_frames = int(simulation_time_analysis)/int(traj_save_freq)
number_frames_analysis = number_frames/int(Skip)


nw_dcd = os.path.join(workDir, str(Jobname) + output_prefix + "_nw." + str(Output_format))
whole_dcd = os.path.join(workDir, str(Jobname) + output_prefix + "_whole." + str(Output_format))
template =  os.path.join(workDir, str(Jobname) + '_%s.dcd')
pdb = os.path.join(workDir, Equilibrated_PDB)

flist = [template % str(i) for i in range(int(first_stride), int(first_stride) + nstride)]

if Remove_waters == "yes":
  #Save topology without waters
  top = pt.load_topology(os.path.join(workDir, "system_amber.prmtop"))
  top_nw = top['!:HOH']
  top_nw.save(os.path.join(workDir, "system_amber_nw.prmtop"))
  # Save trajectory without waters
  trajlist = pt.load(flist, os.path.join(workDir, "system.pdb"), stride=Skip)
  t0 = trajlist.strip(':HOH')
  # traj_image = t0.iterframe(autoimage=True, rmsfit=(0, '!(:H*) & !(:HOH) & !(:NA) & !(:CL) & !(:K) & !(:DLP) & !(:DMP) & !(:DOP)	& !(:DPP) & !(:POP)'))
  traj_image = t0.iterframe(autoimage=True)
  traj_nw = pt.write_traj(nw_dcd, traj_image, overwrite=True, options=Output_format,)
  traj_dcd_check = os.path.exists(nw_dcd)
  traj = nw_dcd
  pdb_ref = os.path.join(workDir, "system_amber_nw.prmtop")
else:
  trajlist = pt.load(flist, os.path.join(workDir, "system.pdb"), stride=Skip)
  traj_image = trajlist.iterframe(autoimage=True)
  traj = pt.write_traj(whole_dcd, traj_image, overwrite=True, options=Output_format)
  traj_dcd_check = os.path.exists(whole_dcd)
  traj = whole_dcd
  pdb_ref = os.path.join(workDir, "system_amber.prmtop")


u = mda.Universe(pdb_ref, traj)
traj_load = pt.load(traj, pdb_ref)
print(traj_load)

apolar1 = u.select_atoms(apolar)
for ts in u.trajectory[0:1:1]:
        with mda.Writer(os.path.join(workDir, 'apolar_' + str(membrane) + '.pdb'), apolar1.n_atoms) as W:
            W.write(apolar1)

polar1 = u.select_atoms(polar)
for ts in u.trajectory[0:1:1]:
        with mda.Writer(os.path.join(workDir, 'polar_' + str(membrane) + '.pdb'), polar1.n_atoms) as W:
            W.write(polar1)

mask_protein = "@CA"
pt_topology = traj_load.top
restraint_array1 = pt.select_atoms(':POP,DLP,DMP,DOP,DPP', pt_topology)
first_atom1 = restraint_array1[0]
last_atom1 = restraint_array1[-1]
mask_membrane = "@" + str(first_atom1+1) + "-" + str(last_atom1+1)
mask_protein_membrane = mask_protein + "," + mask_membrane

if traj_dcd_check == True:
  print("Trajectory concatenated successfully! :-)")
else:
  print("ERROR: Check your inputs! ")

In [ ]:
#@title **Load, view and check the last step of your MD simulation**

u1 = mda.Universe(pdb_ref, traj)
# Write out frames for animation
protein = u1.select_atoms('all')
i = 0
for ts in u1.trajectory[len(u1.trajectory)-1:len(u1.trajectory):1]:
        with mda.Writer('end.pdb', protein.n_atoms) as W:
            W.write(protein)

view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('end.pdb','r').read(),format='pdb')

Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
lip = ['POP','DLP','DMP','DOP','DPP']
view.addStyle({'and':[{'resn':lip}]},
                       {'stick':{'colorscheme':'greenCarbon','radius':0.2}})
view.setViewStyle({'style':'outline','color':'black','width':0.1})
# view.addSurface(py3Dmol.VDW,{'opacity':0.3,'color':'white'})
view.zoomTo()
view.show()

---
---
# **Analysis**

Although visualizing your trajectory can be quite useful, sometimes you also want more quantitative data.

Analyses of MD trajectories vary a lot and we do not intend to cover it all here. However, one can make use of MDanalysis or PyTraj to easily analyze simulations.

Below, you can find a few examples of code snippets that can help you to shed some light on your simulation behavior.

In [ ]:
#@title **Compute density profiles for the subdivided parts of the sytem considering the vector normal to the membrane bilayer (Z axis)**
#@markdown **Provide output file names below:**
Output_name = 'density' #@param {type:"string"}

from scipy.ndimage.filters import gaussian_filter1d
from scipy.interpolate import BSpline

density = pt.density(traj_load, mask=[polar_density, apolar_density, protein_density], density_type='number', delta=0.25, direction='z', dtype='dict',)
head_density = density[polar_density]
tail_density = density[apolar_density]
prot_density = density[protein_density]

head_s = gaussian_filter1d(head_density, sigma=2)
tail_s = gaussian_filter1d(tail_density, sigma=2)
prot_s = gaussian_filter1d(prot_density, sigma=2)
plt.plot(head_s, alpha=1, color = 'limegreen', linewidth = 1.5, label='Membrane Head')
plt.plot(tail_s, alpha=1, color = 'pink', linewidth = 1.5, label='Membrane Tail')
plt.plot(prot_s, alpha=1, color = 'blue', linewidth = 1.5, label='Protein')
plt.legend()
plt.xlim(0,)
plt.ylim(0,)
plt.xlabel("Coordinate [$\AA$]", fontsize = 14, fontweight = 'bold')
plt.ylabel("Density [g/cm$^{-3}$]", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

In [ ]:
#@title **Compute Area per lipid for the membrane**
#@markdown **Provide output file names below:**
Output_name = 'area_per_lipid' #@param {type:"string"}

from lipyphilic.lib.assign_leaflets import AssignLeaflets
from lipyphilic.lib.assign_leaflets import AssignCurvedLeaflets
from lipyphilic.lib.area_per_lipid import AreaPerLipid

# membrane_leaf = u.select_atoms("name P")
# u.trajectory.add_transformations(triclinic_to_orthorhombic(ag=membrane_leaf))

leaflets = AssignCurvedLeaflets(
  universe=u,
  lipid_sel=polar
)
leaflets.run()

# leaflets = AssignLeaflets(
#   universe=u,
#   lipid_sel = polar
# )
# leaflets.run()

areas = AreaPerLipid(
  universe=u,
  lipid_sel=polar,
  leaflets=leaflets.leaflets
)
areas.run()

area_total = 0
for i in range(0,len(areas.areas),1):
  area_total = area_total  + areas.areas[i]
area_per_lipid = (area_total/len(areas.areas))

if int(area_per_lipid[-2]) == int(area_per_lipid[-1]):
  area_total = 0
  for i in range(0,(len(areas.areas)-1),1):
    area_total = area_total  + areas.areas[i]
  area_per_lipid = (area_total/(len(areas.areas)-1))
else:
  pass

area_per_lipid_mean = mean(area_per_lipid)
area_per_lipid_stdev = stdev(area_per_lipid)
print("Area per lipid Average = " + str("{:.2f}".format(area_per_lipid_mean)) + " \u00B1 " + str("{:.2f}".format(area_per_lipid_stdev)) + " Å")

time = len(area_per_lipid)*int(traj_save_freq)/1000
time_array = np.arange(0,time,int(traj_save_freq)/1000)*int(stride_traj)

df = pd.Series(area_per_lipid)
running_aver = df.rolling(window =5).mean()

# Plotting:
ax = plt.plot(time_array, area_per_lipid, alpha=0.3, color = 'limegreen', linewidth = 1.0)
ax = plt.plot(time_array, running_aver, alpha=1, color = 'limegreen', linewidth = 1.2)

plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("Area per lipid [$\AA^2$]", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(area_per_lipid)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Compute Membrane thickness**
#@markdown **Provide output file names below:**
Output_name = 'membrane_thickness ' #@param {type:"string"}

from lipyphilic.lib.memb_thickness import MembThickness
from lipyphilic.lib.assign_leaflets import AssignLeaflets
from lipyphilic.lib.assign_leaflets import AssignCurvedLeaflets


# leaflets = AssignLeaflets(
#   universe=u,
#   lipid_sel = polar
# )
# leaflets.run()

leaflets = AssignCurvedLeaflets(
  universe=u,
  lipid_sel = polar
)
leaflets.run()

memb_thickness = MembThickness(
  universe=u,
  leaflets=leaflets.filter_leaflets(leaf),
  lipid_sel = polar
)
memb_thickness.run()

membrane_thick = memb_thickness.memb_thickness

memb_thickness_mean = mean(memb_thickness.memb_thickness)
memb_thickness_stdev = stdev(memb_thickness.memb_thickness)
print("Membrane Thickness Average = " + str("{:.2f}".format(memb_thickness_mean)) + " \u00B1 " + str("{:.2f}".format(memb_thickness_stdev)) + " Å")

time = len(membrane_thick)*int(traj_save_freq)/1000
time_array = np.arange(0,time,int(traj_save_freq)/1000)*int(stride_traj)

df = pd.Series(membrane_thick)
running_aver = df.rolling(window =5).mean()

# Plotting:
ax = plt.plot(time_array, membrane_thick, alpha=0.3, color = 'limegreen', linewidth = 1.0)
ax = plt.plot(time_array, running_aver, alpha=1, color = 'limegreen', linewidth = 1.2)

plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("Membrane Thickness [$\AA$]", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(membrane_thick)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Compute Lateral Diffusion for membrane (Mean square displacement)**
#@markdown **Provide output file names below:**
Output_name = 'lateral_diffusion ' #@param {type:"string"}

from lipyphilic.lib.lateral_diffusion import MSD

msd = MSD(
  universe=u,
  lipid_sel=polar
)

msd.run()
msd_total = 0
for i in range(0,len(msd.msd),1):
  msd_i = msd.msd[i]
  msd_total = msd_total  + msd_i
msd_end = (msd_total/len(msd.msd))

msd_end_mean = mean(msd_end)
msd_end_stdev = stdev(msd_end)
print("Lateral Diffusion Average = " + str("{:.2f}".format(msd_end_mean)) + " \u00B1 " + str("{:.2f}".format(msd_end_stdev)) + " Å")

time = len(msd_end)*int(traj_save_freq)/1000
time_array = np.arange(0,time,int(traj_save_freq)/1000)*int(stride_traj)

df = pd.Series(msd_end)
running_aver = df.rolling(window =5).mean()

# Plotting:
ax = plt.plot(time_array, msd_end, alpha=0.3, color = 'limegreen', linewidth = 1.0)
ax = plt.plot(time_array, running_aver, alpha=1, color = 'limegreen', linewidth = 1.2)

plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("MSD [$\AA^2$]", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(msd_end)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Compute hydrogen bonds**

Selection_1 = "Protein" #@param ["Protein", "Membrane"]
Selection_2 = "Membrane" #@param ["Protein", "Membrane"]

pt_topology = traj_load.top
restraint_array1 = pt.select_atoms('!(:HOH) & !(:NA) & !(:CL) & !(:MG) & !(:K) & !(:POP,DLP,DMP,DOP,DPP)', pt_topology)
first_atom1 = restraint_array1[0]
last_atom1 = restraint_array1[-1]
mask_protein_hbond = "@" + str(first_atom1+1) + "-" + str(last_atom1+1)

pt_topology = traj_load.top
restraint_array1 = pt.select_atoms(':POP,DLP,DMP,DOP,DPP', pt_topology)
first_atom1 = restraint_array1[0]
last_atom1 = restraint_array1[-1]
mask_membrane_hbond = "@" + str(first_atom1+1) + "-" + str(last_atom1+1)

if Selection_1 == "Protein"  and Selection_2 == "Protein":
  mask_end = mask_protein_hbond
  mask_hbond1 = 'acceptormask ' + mask_end + ' donormask ' + mask_end
  hb1 = pt.hbond(traj_load, options=mask_hbond1, dtype='dict', distance =3.5)
  hb = hb1['total_solute_hbonds']
  # print(mask_end)
elif Selection_1 == "Membrane"  and Selection_2 == "Membrane":
  mask_end = mask_membrane_hbond
  mask_hbond1 = 'acceptormask ' + mask_end + ' donormask ' + mask_end
  hb1 = pt.hbond(traj_load, options=mask_hbond1, dtype='dict', distance =3.5)
  hb = hb1['total_solute_hbonds']
  # print(mask_end)
else:
  mask_end1 = mask_protein_hbond
  mask_end2 = mask_membrane_hbond
  mask_hbond1 = 'acceptormask ' + mask_end1 + ' donormask ' + mask_end2
  mask_hbond2 = 'acceptormask ' + mask_end2 + ' donormask ' + mask_end1

  hb1 = pt.hbond(traj_load, options=mask_hbond1, dtype='dict', distance =3.5)
  hb2 = pt.hbond(traj_load, options=mask_hbond2, dtype='dict', distance =3.5)

  hb_end1 = hb1['total_solute_hbonds']
  hb_end2 = hb2['total_solute_hbonds']
  hb = hb_end1 + hb_end2
  # print(mask_end1,mask_end2)

#@markdown **Provide output file names below:**
Output_name = 'hbond_inter' #@param {type:"string"}

time = len(hb)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)

df = pd.Series(hb)
running_aver = df.rolling(window =5).mean()

# hb.keys()
# print(hb_total)
# hb_end_mean = mean(hb)
# hb_end_stdev = stdev(hb)
# print("Hydrogen Bonds Average = " + str("{:.2f}".format(hb_end_mean)) + " \u00B1 " + str("{:.2f}".format(hb_end_stdev)))

# Plotting:
ax = plt.plot(time_array, hb, alpha=0.2, color = 'black', linewidth = 1.0)
ax = plt.plot(time_array, running_aver, alpha=1, color = 'black', linewidth = 1.0)

plt.xlim(0, simulation_ns)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("Hydrogen Bonds", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(hb)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **2D RMSD**

Selection = "Protein+Membrane" #@param ["Protein", "Protein+Membrane", "Membrane"]

if Selection == "Protein":
  mask_end = mask_protein
elif Selection == "Protein+Membrane":
  mask_end = mask_protein_membrane
else:
  mask_end = mask_membrane


#@markdown **Provide output file names below:**

Output_name = '2D_rmsd' #@param {type:"string"}

last_frame = len(time_array)

stride_ticks_f = (last_frame)/5
ticks_frame = np.arange(0,(len(time_array) + float(stride_ticks_f)), float(stride_ticks_f))
a = ticks_frame.astype(float)
stride_ticks_t = (simulation_ns)/5
tick_time = np.arange(0,(float(simulation_ns) + float(stride_ticks_t)), float(stride_ticks_t))
b = tick_time.astype(float)

mat1 = pt.pairwise_rmsd(traj_load, mask=mask_end, frame_indices=range(int(number_frames_analysis)))


ax = plt.imshow(mat1, cmap = 'PRGn', origin='lower', interpolation = 'bicubic')
plt.title('2D RMSD')
plt.xlabel('Time (ns)', fontsize = 14, fontweight = 'bold')
plt.ylabel('Time (ns)', fontsize = 14, fontweight = 'bold')
# plt.xticks(fontsize = 12)
# plt.yticks(fontsize = 12)
plt.xticks(a, b.round(decimals=3), fontsize = 12)
plt.yticks(a, b.round(decimals=3), fontsize = 12)
# plt.xlim(0, a[-1])
# plt.ylim(0, a[-1])

cbar1 = plt.colorbar()
cbar1.set_label("RMSD ($\AA$)", fontsize = 14, fontweight = 'bold')


plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(mat1)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Compute distance between the residues**
#@markdown **Provide output file names below:**
Output_name = 'distance_residues' #@param {type:"string"}
#@markdown **Type the residues numbers separated by commas and without spaces (1,2,3...):**
Selection_1 = '3' #@param {type:"string"}
Selection_2 = '12' #@param {type:"string"}

mask = ":" + str(Selection_1) + " :" + str(Selection_2)
dist = pt.distance(traj_load, mask)
print("Selected residues = " + Selection_1 + ", "  + Selection_2 + "\n")
dist_mean = mean(dist)
dist_stdev = stdev(dist)
print("Distance Average = " + str("{:.2f}".format(dist_mean)) + " \u00B1 " + str("{:.2f}".format(dist_stdev)) + " Å")

time = len(dist)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)

df = pd.Series(dist)
running_aver = df.rolling(window =5).mean()

# Plotting:
ax = plt.plot(time_array, dist, alpha=0.2, color = 'magenta', linewidth = 1.0)
ax = plt.plot(time_array, running_aver, alpha=1, color = 'magenta', linewidth = 1.0)

plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("Distance [$\AA$]", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(dist)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Plot distance as a distribution**

#@markdown **Provide output file names below:**
Output_name = 'distance_dist' #@param {type:"string"}

ax = sb.kdeplot(dist, color="magenta", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('Distance [$\AA$]', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

In [ ]:
#@title **Compute RMSD (protein)**

#@markdown **Provide output file names below:**

Output_name = 'rmsd' #@param {type:"string"}

rmsd = pt.rmsd(traj_load, ref = 0, mask = mask_protein)

time = len(rmsd)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)

df = pd.Series(rmsd)
running_aver = df.rolling(window =5).mean()

# Plotting:
ax = plt.plot(time_array, rmsd, alpha=0.2, color = 'blue', linewidth = 1.0)
ax = plt.plot(time_array, running_aver, alpha=1, color = 'blue', linewidth = 1.0)

plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("RMSD [$\AA$]", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(rmsd)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Plot RMSD as a distribution**

#@markdown **Provide output file names below:**
Output_name = 'rmsd_dist' #@param {type:"string"}

ax = sb.kdeplot(rmsd, color="blue", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('RMSD [$\AA$]', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

In [ ]:
#@title **Compute radius of gyration (protein)**

#@markdown **Provide output file names below:**

Output_name = 'radius_gyration' #@param {type:"string"}

radgyr = pt.radgyr(traj_load, mask = mask_protein)
time = len(rmsd)*int(Write_the_trajectory)/1000
time_array = np.arange(0,time,int(Write_the_trajectory)/1000)*int(stride_traj)

df = pd.Series(radgyr)
running_aver = df.rolling(window =5).mean()

# Plotting:
plt.plot(time_array, radgyr, alpha=0.2, color = 'green', linewidth = 1.0)
plt.plot(time_array, running_aver, alpha=1, color = 'green', linewidth = 1.0)

plt.xlim(0, simulation_ns)
#plt.ylim(2, 6)

plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("Radius of gyration ($\AA$)", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(radgyr)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Plot radius of gyration as a distribution**

#@markdown **Provide output file names below:**
Output_name = 'radius_gyration_dist' #@param {type:"string"}

ax = sb.kdeplot(radgyr, color="green", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('Radius of gyration ($\AA$)', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

In [ ]:
#@title **Compute RMSF (protein)**

#@markdown **Provide output file names below:**

Output_name = 'rmsf' #@param {type:"string"}

traj_align = pt.align(traj_load, mask=mask_protein, ref=0)
rmsf = pt.rmsf(traj_align, mask = mask_protein)
# bfactor = pt.bfactors(traj_load, byres=False)

# Plotting:
plt.plot(rmsf[:,1], alpha=1.0, color = 'red', linewidth = 1.0)

plt.xlabel("Residue", fontsize = 14, fontweight = 'bold')
plt.ylabel("RMSF ($\AA$)", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.xlim(0, len(rmsf[:-1]))

#plt.xticks(np.arange(min(rmsf[:1]), max(rmsf[:1])))
plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(rmsf)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))

In [ ]:
#@title **Calculate eigenvectors of Principle Component Analysis (PCA) for proteins**

data = pt.pca(traj_load, fit=True, ref=0, mask=mask_protein, n_vecs=2)
#print('projection values of each frame to first mode = {} \n'.format(data[0][0]))
#print('projection values of each frame to second mode = {} \n'.format(data[0][1]))
#print('eigvenvalues of first two modes', data[1][0])
#print("")
#print('eigvenvectors of first two modes: \n', data[1][1])

last_frame = len(time_array)

stride_ticks_f = (last_frame)/5
ticks_frame = np.arange(0,(len(time_array) + float(stride_ticks_f)), float(stride_ticks_f))
a = ticks_frame.astype(float)
a2 = a.tolist()
stride_ticks_t = (simulation_ns)/5
tick_time = np.arange(0,(float(simulation_ns) + float(stride_ticks_t)), float(stride_ticks_t))
b = tick_time.astype(float)

#@markdown **Provide output file names below:**
Output_name = 'PCA' #@param {type:"string"}

Output_PC1 = 'PC1' #@param {type:"string"}
Output_PC2 = 'PC2' #@param {type:"string"}

%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # high resolution
projection_data = data[0]
plt.title(r'PCA')
PC1 = data[0][0]
PC2 = data[0][1]

a = plt.scatter(PC1,PC2, c=range(int(number_frames_analysis)), cmap='Greens', marker='o',s=8, alpha=1)
plt.clim(0, last_frame)

plt.xlabel('PC1', fontsize = 14, fontweight = 'bold')
plt.ylabel('PC2', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
# N = len(number_frames)
# x2 = np.arange(N)

cbar1 = plt.colorbar(a, orientation="vertical")
cbar1.set_label('Time(ns)', fontsize = 14, fontweight = 'bold')
cbar1.set_ticks(a2)
cbar1.set_ticklabels(b.round(decimals=3))

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

pc1=pd.DataFrame(PC1)
pc1.to_csv(os.path.join(workDir, Output_PC1 + ".csv"))
pc2=pd.DataFrame(PC2)
pc2.to_csv(os.path.join(workDir, Output_PC2 + ".csv"))

In [ ]:
#@title **Plot Principal Component 1 (PC1) and Principal Component 2 (PC2) as a distribution**
Output_name = 'PCA_dist' #@param {type:"string"}


fig = plt.figure(figsize=(9,5))

plt.subplot(1, 2, 1)
ax = sb.kdeplot(PC1, color="green", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('PC1', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)
plt.subplot(1, 2, 2)
ax2 = sb.kdeplot(PC2, color="purple", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('PC2', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['bottom'].set_visible(True)
ax2.spines['left'].set_visible(False)


plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

In [ ]:
#@title **Pearson's Cross Correlation (CC) for protein**

#@markdown **Provide output file names below:**

Output_name = 'cross_correlation' #@param {type:"string"}


traj_align = pt.align(traj_load, mask=mask_protein, ref=0)

mat_cc = matrix.correl(traj_align, mask_protein)

ax = plt.imshow(mat_cc, cmap = 'PiYG_r', interpolation = 'bicubic', vmin = -1, vmax = 1, origin='lower')

plt.xlabel('Residues', fontsize = 14, fontweight = 'bold')
plt.ylabel('Residues', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
cbar1 = plt.colorbar()
cbar1.set_label('$CC_ij$', fontsize = 14, fontweight = 'bold')

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(mat_cc)
raw_data.to_csv(os.path.join(workDir, Output_name + ".csv"))